In [5]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

from importlib import reload

from functools import partial

import os
from os.path import join
import subprocess
import time
import sys
sys.path.append('../../') # lets us import ptype package from the subdir
#import ptype.

from joblib import Parallel, delayed

from xhistogram.xarray import histogram


import soundings.utils as sounding_utils
import batch_jobs.xr_map_reduce as xmr

In [2]:
reload(xmr)
dirpaths = ['/glade/scratch/dkimpara/ptype_case_studies/kentucky/rap/20220224/0000',
            '/glade/scratch/dkimpara/ptype_case_studies/kentucky/rap/20220224/0600',
           '/glade/scratch/dkimpara/ptype_case_studies/kentucky/rap/20220223/0000',
           '/glade/scratch/dkimpara/ptype_case_studies/kentucky/rap/20220223/0600']
    
results = Parallel(n_jobs=-1)(delayed(xmr.xr_map)(path, xmr.compute_func) for path in dirpaths)



In [3]:
ds = results[0]
ds

<xarray.Dataset>
Dimensions:            (predtype: 8, case_study_day: 1, time: 1, step: 18,
                        heightAboveGround: 21, bin: 999)
Coordinates:
  * predtype           (predtype) object 'ML_cicep' 'cicep' ... 'crain'
  * case_study_day     (case_study_day) object 'kentucky-20220224'
  * time               (time) int64 1645660800
  * step               (step) float64 1.0 2.0 3.0 4.0 ... 15.0 16.0 17.0 18.0
  * heightAboveGround  (heightAboveGround) int64 0 250 500 ... 4500 4750 5000
  * bin                (bin) float64 -59.95 -59.85 -59.75 ... 39.65 39.75 39.85
Data variables:
    num_obs            (predtype, case_study_day, step, time) int64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    t_h_fabv           (predtype, case_study_day, step, time) float64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    dpt_h_fabv         (predtype, case_study_day, step, time) float64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    wb_h_fabv          (predtype, case_study_day, step, time) float64 dask.array<chunksize=(1, 1, 1, 1), meta=np.ndarray>
    t_h_mean           (predtype, case_study_day, step, time, heightAboveGround) float32 dask.array<chunksize=(1, 1, 1, 1, 21), meta=np.ndarray>
    dpt_h_mean         (predtype, case_study_day, step, time, heightAboveGround) float32 dask.array<chunksize=(1, 1, 1, 1, 21), meta=np.ndarray>
    wb_h_mean          (predtype, case_study_day, step, time, heightAboveGround) float32 dask.array<chunksize=(1, 1, 1, 1, 21), meta=np.ndarray>
    t_h_hist           (predtype, case_study_day, step, time, heightAboveGround, bin) float64 dask.array<chunksize=(1, 1, 1, 1, 21, 999), meta=np.ndarray>
    dpt_h_hist         (predtype, case_study_day, step, time, heightAboveGround, bin) float64 dask.array<chunksize=(1, 1, 1, 1, 21, 999), meta=np.ndarray>
    wb_h_hist          (predtype, case_study_day, step, time, heightAboveGround, bin) float64 dask.array<chunksize=(1, 1, 1, 1, 21, 999), meta=np.ndarray>
    valid_time         (case_study_day, step) float64 1.646e+09 ... 1.646e+09

In [23]:
rinithr = map(time_to_inithr, results)

In [24]:
res = xr.merge(rinithr)

In [25]:
res

<xarray.Dataset>
Dimensions:            (predtype: 8, case_study_day: 2, init_hr: 2, step: 18,
                        heightAboveGround: 21, bin: 999)
Coordinates:
  * predtype           (predtype) object 'ML_cicep' 'cicep' ... 'crain'
  * case_study_day     (case_study_day) object 'kentucky-20220223' 'kentucky-...
  * init_hr            (init_hr) int64 0 6
  * step               (step) float64 1.0 2.0 3.0 4.0 ... 15.0 16.0 17.0 18.0
  * heightAboveGround  (heightAboveGround) int64 0 250 500 ... 4500 4750 5000
  * bin                (bin) float64 -59.95 -59.85 -59.75 ... 39.65 39.75 39.85
Data variables:
    num_obs            (predtype, case_study_day, step, init_hr) float64 dask.array<chunksize=(1, 2, 1, 2), meta=np.ndarray>
    t_h_fabv           (predtype, case_study_day, step, init_hr) float64 dask.array<chunksize=(1, 2, 1, 2), meta=np.ndarray>
    dpt_h_fabv         (predtype, case_study_day, step, init_hr) float64 dask.array<chunksize=(1, 2, 1, 2), meta=np.ndarray>
    wb_h_fabv          (predtype, case_study_day, step, init_hr) float64 dask.array<chunksize=(1, 2, 1, 2), meta=np.ndarray>
    t_h_mean           (predtype, case_study_day, step, init_hr, heightAboveGround) float32 dask.array<chunksize=(1, 2, 1, 2, 21), meta=np.ndarray>
    dpt_h_mean         (predtype, case_study_day, step, init_hr, heightAboveGround) float32 dask.array<chunksize=(1, 2, 1, 2, 21), meta=np.ndarray>
    wb_h_mean          (predtype, case_study_day, step, init_hr, heightAboveGround) float32 dask.array<chunksize=(1, 2, 1, 2, 21), meta=np.ndarray>
    t_h_hist           (predtype, case_study_day, step, init_hr, heightAboveGround, bin) float64 dask.array<chunksize=(1, 2, 1, 2, 21, 999), meta=np.ndarray>
    dpt_h_hist         (predtype, case_study_day, step, init_hr, heightAboveGround, bin) float64 dask.array<chunksize=(1, 2, 1, 2, 21, 999), meta=np.ndarray>
    wb_h_hist          (predtype, case_study_day, step, init_hr, heightAboveGround, bin) float64 dask.array<chunksize=(1, 2, 1, 2, 21, 999), meta=np.ndarray>
    valid_time         (init_hr, case_study_day, step) float64 1.646e+09 ... ...

In [ ]:
res = xr.merge(results)

savepath = f'/glade/work/dkimpara/ptype-aggs/test.nc'
subprocess.run(f"rm {savepath}", shell=True, encoding='utf-8')
res.to_netcdf(savepath)


In [5]:
sounding_utils.count_nulls(res)

0
0
0
0
0
0
0
0
0
0
0
